<span style=" color: yellow; font-size: 24px;">Deep Learning 1 Project _ Group 5:<br>Tweet Sentiment Extraction (Kaggle · Featured Code Competition)<br></span><span style="font-size: 22px;">
Shrey	Patel	101541370<br>
Sam	Emami	101575471<br>
Eric	Lessa	101549935<br>
Dwip	Makwana	101483523<br>
Moossa	Hussain	101542820<br>
Chaoyu	Liu	101573622<br>
Devanshi 	Dave	101582208<br>
Rutika	Bhuva	101551781<br>
</span>

<span style=" color: yellow; font-size: 24px;">Use This Template **by memeber name**: </span>description for each part<span style="font-size: 22px;">

In [ ]:
#!pip install shap
#!pip install transformers
#!pip install pipeline
#go to https://graphviz.gitlab.io/download/ and install graphviz

In [2]:
!pip install datasets

In [3]:
!pip show torchvision


Name: torchvision
Version: 0.20.1+cu118
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: c:\users\dwipm\.conda\envs\ml-venv-310\lib\site-packages
Requires: numpy, pillow, torch
Required-by: 


<span style="color: yellow; font-size: 24px;">Hugging Face Transformers Tools for Using Pre-trained LLMs:<br><br></span>
<span style="color: yellow; font-size: 24px;">pipeline: </span><span style="font-size: 22px;"> It is like a convenient wrapper or helper that simplifies the process of working with models such as those loaded with AutoModelForSequenceClassification and their associated tokenizers.<br>It integrates with pre-trained models and tokenizers for specific tasks and perform NLP tasks and the processes such as:<br>
Loading the model.<br>
Tokenizing the input text.<br>
Generating predictions.<br>
Post-processing the output (if necessary)<br><br></span>
<span style="color: yellow; font-size: 24px;">AutoModelForSequenceClassification: </span><span style="font-size: 22px;"> Loading Pre-trained Models designed for sequence classification tasks. Sequence Classification refers to tasks where you classify an entire sequence of text (like a sentence or document) into a category. Examples include:<br>
<span style="color: red">Sentiment analysis (positive, negative, neutral)</span><br>
Topic classification (news, sports, politics)<br>
Spam detection<br>
Intent classification (e.g., order, question, complaint)<br>
"Auto" in the Name: The Auto prefix in the class name signifies its flexibility. You can use it to load various pre-trained models from different architectures (BERT, RoBERTa, DistilBERT, etc.) simply by specifying the model name or path.<br><br></span>
<span style="color: yellow; font-size: 24px;">AutoTokenizer: </span><span style="font-size: 22px;"> tokenizing the inputs. The tokenizer ensures that:<br>
The tokenized input matches the vocabulary and format expected by the model.<br>
The inputs include features like input_ids, attention_mask, and token_type_ids (if applicable).<br><br></span>

In [4]:
import pandas as pd
import shap
import numpy as np
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer


In [5]:

# Load the datasets
train_df = pd.read_csv('train.csv')
#test_df = pd.read_csv('test.csv')


In [6]:
# Display the first few rows of the training data
print("Training Data Sample:\n")
print(train_df.head())

Training Data Sample:

       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment  
0  I`d have responded, if I were going   neutral  
1                             Sooo SAD  negative  
2                          bullying me  negative  
3                       leave me alone  negative  
4                        Sons of ****,  negative  


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [8]:
train_df = train_df.dropna(subset=['text'])

In [9]:
print('max tweet:', train_df['text'].apply(len).max())


max tweet: 141


<span style="font-size: 22px;">From the Hugging Face Transformers library, acquires a pre-trained BERT-based sentiment analysis model (**LLM**), with its appropriate pipeline (**classifier**) and **tokenizer**.</span>


In [10]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

model = AutoModelForQuestionAnswering.from_pretrained('bert-base-uncased')

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from datasets import Dataset

dataset = Dataset.from_pandas(train_df)

In [12]:
train_dataset = dataset.train_test_split(test_size=0.1)['train']
eval_dataset = dataset.train_test_split(test_size=0.1)['test']

In [13]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to device
model = model.to(device)

Using device: cuda


In [14]:
def prepare_train_features(examples):
    sentiment = examples['sentiment']
    tweet = examples['text']
    span = examples['selected_text']

    tokenized_qa = tokenizer(sentiment, # question
                             tweet, # context
                             padding='max_length',
                             return_offsets_mapping=True).to(device)

    qa_tokens = tokenized_qa["input_ids"]

    span_tokens = tokenizer(span)["input_ids"]

    tokenized_qa["start_positions"] = []
    tokenized_qa["end_positions"] = []

    start_char = tweet.find(span)
    end_char = start_char + len(span)

    offsets = tokenized_qa.pop("offset_mapping")
    start_token = end_token = None
    for idx, (start, end) in enumerate(offsets):
      if start <= start_char < end:
        start_token = idx
      if start < end_char <= end:
        end_token = idx
        break

    tokenized_qa["start_positions"].append(start_token)
    tokenized_qa["end_positions"].append(end_token)

    if start is None or end is None:
      print(tweet + ' -> ' + span)
      print(f' {qa_tokens} -> {span_tokens}')
      return None

    return tokenized_qa


In [15]:
tokenized_train_dataset = train_dataset.map(prepare_train_features)


Map:   0%|          | 0/24732 [00:00<?, ? examples/s]

In [16]:
tokenized_eval_dataset = eval_dataset.map(prepare_train_features)

Map:   0%|          | 0/2748 [00:00<?, ? examples/s]

In [17]:
tokenized_train_dataset

Dataset({
    features: ['textID', 'text', 'selected_text', 'sentiment', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 24732
})

In [18]:
tokenized_eval_dataset

Dataset({
    features: ['textID', 'text', 'selected_text', 'sentiment', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 2748
})

In [19]:
small_train_dataset = tokenized_train_dataset.shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_eval_dataset.shuffle(seed=42).select(range(1000))

In [20]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"finetune-BERT-tweet",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    # GPU specific settings
    no_cuda=False,
    fp16=True,  # Enable mixed precision training
    fp16_opt_level="O1",  # Mixed precision optimization level
    fp16_backend="auto",
    dataloader_num_workers=4,  # Increase parallel data loading
)

c:\Users\dwipm\.conda\envs\ml-venv-310\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [25]:

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

C:\Users\dwipm\AppData\Local\Temp\ipykernel_23284\735706203.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [24]:
import torch

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.614041
2,No log,1.361142
3,No log,1.339347


TrainOutput(global_step=375, training_loss=1.6745960286458332, metrics={'train_runtime': 201.0934, 'train_samples_per_second': 14.918, 'train_steps_per_second': 1.865, 'total_flos': 783890270208000.0, 'train_loss': 1.6745960286458332, 'epoch': 3.0})

### Predictions sentiment extraction

In [49]:
from transformers import AutoModelForTokenClassification, AutoTokenizer


model = AutoModelForQuestionAnswering.from_pretrained("/kaggle/working/finetune-BERT-tweet/checkpoint-375")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/finetune-BERT-tweet/checkpoint-375")



In [50]:
tweet = " you guys didn`t say hi or answer my questions yesterday  but nice songs."
sentiment = "positive"

inputs = tokenizer(
    sentiment,  # the sentiment question
    tweet,      # the tweet context
    padding=True,
    truncation=True,
    return_tensors="pt"
)


In [51]:
import torch

model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    start_position = torch.argmax(start_logits, dim=-1)
    end_position = torch.argmax(end_logits, dim=-1)

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    
    sentiment_span = tokens[start_position:end_position+1]
    sentiment_text = tokenizer.convert_tokens_to_string(sentiment_span)
    
    print(f"Predicted sentiment span: {sentiment_text}")


Predicted sentiment span: nice


<span style=" color: yellow; font-size: 24px;">Importan Note by Sam: </span><span style="font-size: 22px;">LLMs such as BERT, LLama, GPTT,etc., and BERT-based models (like distilbert-base-uncased-finetuned-sst-2-english) do not include an activation function (e.g., softmax or sigmoid) at their output layer by default. Instead, they output logits, which are raw, unnormalized scores for each class. The activation function, if needed, is applied externally depending on the specific task and requirements.<br> BERT is a general-purpose transformer model. By omitting the activation function, it can be fine-tuned for various tasks: Multi-class classification (e.g., softmax), Binary classification (e.g., sigmoid), Regression tasks (no activation function).<br></span>


<span style=" color: yellow; font-size: 24px;">outputs</span> <span style="font-size: 22px;">It is an object of type ModelOutput, specifically tailored for the model we are using. Here we didn't simply print the result of the sentiment for the each tweet, but we used output.logits to go access the detailed classification results for each text for further word by word analysis in SHAP.<br></span>
<span style=" color: yellow; font-size: 24px;">logits:</span> <span style="font-size: 22px;"> is the model’s raw output before applying the activation function. logits are unnormalized scores, which can be positive, negative, or zero, depending on the model’s confidence in each class. Here "outputs.logits" is a tensor containing the raw, unnormalized predictions for each input sequence. Each row in the tensor corresponds to one input in texts, and each column corresponds to a class.<br></span>
<span style=" color: yellow; font-size: 24px;">Normalize logits to probabilities:</span><span style="font-size: 22px;"> by applying softmax we normalize the raw predictions for each tweet to probabilities (the results are printed for clarity).<br></span>
<span style=" color: yellow; font-size: 24px;">detach()</span><span style="font-size: 22px;"> We use it since we don’t want to compute gradients for the tensor. Since probabilities are used only for prediction (not training), there’s no need to track gradients, so detach() avoids unnecessary computational overhead.<br></span>


In [53]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
# Load pre-trained sentiment analysis model and tokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
classifier = pipeline("sentiment-analysis", model=model_name)

# Tokenizer and model for SHAP
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Device set to use cuda:0


In [54]:
# Define a wrapper for the classifier to work with SHAP
def classifier_wrapper(texts):
    tokens = tokenizer(list(texts), padding=True, truncation=True, return_tensors="pt")
    outputs = model(**tokens)
    probabilities = outputs.logits.softmax(dim=1).detach().numpy()
    print(f"\nlogits contains tweets' unnormalized prediction scores for each class(positive / negative):\n{outputs.logits}\n")
    return probabilities

# Select a subset of the training data for SHAP analysis
sample_texts = train_df["text"].sample(10, random_state=42).tolist()
print("Sample Texts:")
print(sample_texts)
print(f"Probibilities after normalizing logits by softwax:\n {classifier_wrapper(sample_texts)}")  # Should output a numpy array of probabilities


Sample Texts:
[' Enjoy! Family trumps everything', ' --of them kinda turns me off of it all.  And then I buy more of them and dig a deeper hole, etc. ;;', 'Clive it`s my birthday pat me  http://apps.facebook.com/dogbook/profile/view/6386106', ' congrats hey', 'is texting', ' Do you have any idea when the (not so) patient fans will see some teaser pics of you all in costume?', ' Tell him where...', ' Ooooh, I`m jealous  I might try and get some for the saturday but I have an exam on the monday that Im gonna fail', ' OHSHNAPSSS. is she pissed at blair as usual ? hahah. & yeeeah, i bake cookies', 'wee. done with advance audit paper']

logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[-4.3473,  4.6681],
        [ 3.1155, -2.5241],
        [ 2.4987, -2.0141],
        [-3.6120,  3.9220],
        [ 0.7955, -0.6091],
        [ 3.1666, -2.6283],
        [ 1.3965, -1.2101],
        [ 3.7944, -3.1330],
        [ 3.2341, -2.6566],
        [ 2.1967

<span style=" color: yellow; font-size: 24px;">SHAP Analysis:</span><span style="font-size: 22px;"> SHAP is based on <u>Shapley values</u> which is a concept from <u>Game Theory</u>. Imagine a group of players collaborating to achieve a common goal. Shapley values determine how to fairly distribute the rewards or profits among the players based on their individual contributions.
In the context of machine learning, features of a data point can be seen as "players" contributing to the model's prediction.<br> SHAP aims to fairly attribute the "credit" (or "blame") for the model's output to each feature.<br>
For complex models, SHAP often uses <u>approximations</u> or <u>sampling techniques</u> to estimate Shapley values.<br>
The explainer considers <u>all possible combinations of features (words or tokens)</u> in the sample.
For each combination, it calculates the model's prediction.<br>
This process helps determine how much each feature contributes to the overall prediction.
Shapley Values are assigned based on the calculated predictions for different feature combinations,
SHAP assigns a value to each feature.
</br></br></spn>
</span>
<span style=" color: yellow; font-size: 24px;">shap.Explainer:</span><span style="font-size: 22px;"> Is the core class in the SHAP library. It encapsulates the model and the method for <u>calculating</u> SHAP values, which quantify the <u>contribution of each feature</u> (in this case, words or tokens) <u>to the model's prediction</u> for a given sample.<br>
The Explainer needs "classifier_wrapper" function to understand how the model makes predictions.<br><br></span>

<span style=" color: yellow; font-size: 24px;">Note: </span><span style="font-size: 22px;">
When classifier_wrapper is run by SHAP, there are six or less probibilities for the ten tweets in sample_texts. It is because the tokenizer applies padding and truncation to the input. If multiple texts are identical, the tokenizer may return the same tokens for duplicates. If sample_texts contains duplicate or very similar tweets, their logits will be identical, and we might get fewer output tensors.<br><br></span>

In [55]:

# Explain predictions using SHAP
# This line integrates the model in the function to SHAP and encapsulates
# the necessary information to compute SHAP values for the given model and data.
explainer = shap.Explainer(classifier_wrapper, tokenizer)
shap_values = explainer(sample_texts)

# Visualize the SHAP values for each word:
# Output 0 and Output 1: predicted probabilities for two classes
for i, text in enumerate(sample_texts):
    print(f"\nSHAP Analysis for Text {i + 1}: {text}")
    shap.plots.text(shap_values[i])
'''
#The following code save SHAP analysis results into different html files for each tweet:

#shap.save_html("shap_analysis.html", shap_values)
# Manually set the base value if explainer.expected_value is None
# For classification tasks, the base value can be the mean of logits
if explainer.expected_value is None:
    print("explainer.expected_value is None. Computing a manual base value.")
    base_value = np.mean([np.mean(values.values) for values in shap_values])
else:
    base_value = explainer.expected_value

# Loop through all SHAP values and save each as a separate force plot
for i, shap_value in enumerate(shap_values):
    # Debugging: Print base value and SHAP values
    print(f"Processing Text {i + 1}")
    print(f"Base Value: {base_value}")
    print(f"SHAP Values for Text {i + 1}: {shap_value.values}")

    # Check if shap_value.values is None
    if shap_value.values is None:
        print(f"Skipping Text {i + 1} due to missing values.")
        continue

    # Handle multi-output models (if applicable)
    if isinstance(base_value, list):
        base_value = base_value[0]  # Use the base value for the first output

    # Create a force plot for the current sample
    try:
        vis = shap.plots.force(base_value, shap_value.values)

        # Save the visualization as an HTML file
        file_name = f"shap_analysis_{i + 1}.html"
        shap.save_html(file_name, vis)
        print(f"Saved SHAP visualization for Text {i + 1} as {file_name}")
    except Exception as e:
        print(f"Error processing Text {i + 1}: {e}")
'''


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[ 0.4748, -0.3312]], grad_fn=<AddmmBackward0>)


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[-4.3473,  4.6681]], grad_fn=<AddmmBackward0>)


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[-4.2475,  4.5537],
        [-2.8640,  2.9712]], grad_fn=<AddmmBackward0>)


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[-4.0612,  4.3733],
        [-1.6489,  1.7447],
        [-1.6678,  1.8224],
        [-1.2598,  1.3890],
        [-4.2308,  4.5345],
        [-3.3864,  3.5596]], grad_fn=<AddmmBackward0>)


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[ 0.4748, -0.3312],
        [-4.0612,  4.3733],
        [-1.1990,  1.2813],
        [ 0.3021,  0.0121],
        [-1.2598,  1.3890],
   

  0%|          | 0/498 [00:00<?, ?it/s]


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[ 1.2397, -1.1105],
        [ 1.1909, -1.0626],
        [ 0.5336, -0.4654],
        [ 0.3986, -0.3121],
        [ 0.6193, -0.3398],
        [ 1.8549, -1.4929]], grad_fn=<AddmmBackward0>)


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[ 0.4177, -0.3161],
        [ 0.0998, -0.0153],
        [ 1.2861, -0.9367],
        [ 1.4912, -1.1388],
        [ 1.3811, -1.2438],
        [ 1.3855, -1.2465]], grad_fn=<AddmmBackward0>)


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[ 0.9979, -0.8808],
        [ 1.2131, -1.0679],
        [ 1.8820, -1.5385],
        [ 1.8141, -1.4805],
        [ 1.2241, -1.1129],
        [ 1.3864, -1.2362]], grad_fn=<AddmmBackward0>)


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[ 1.1287, -0.9943],
        [ 1.

PartitionExplainer explainer:  20%|██        | 2/10 [00:00<?, ?it/s]


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[-0.0353,  0.0853]], grad_fn=<AddmmBackward0>)


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[ 2.4987, -2.0141]], grad_fn=<AddmmBackward0>)


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[ 1.2439, -0.9198],
        [ 1.4507, -1.2518]], grad_fn=<AddmmBackward0>)


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[ 0.6787, -0.5620],
        [ 1.4905, -1.3115],
        [-1.0822,  1.2324],
        [ 0.6651, -0.3936],
        [ 1.7019, -1.3649],
        [ 1.9886, -1.5925]], grad_fn=<AddmmBackward0>)


logits contains tweets' unnormalized prediction scores for each class(positive / negative):
tensor([[ 0.3235, -0.0310],
        [-0.3155,  0.5216],
        [ 1.4310, -1.2554],
        [ 0.7324, -0.6190],
        [ 0.1045,  0.0026],
   

...


SHAP Analysis for Text 7:  Tell him where...



SHAP Analysis for Text 8:  Ooooh, I`m jealous  I might try and get some for the saturday but I have an exam on the monday that Im gonna fail



SHAP Analysis for Text 9:  OHSHNAPSSS. is she pissed at blair as usual ? hahah. & yeeeah, i bake cookies



SHAP Analysis for Text 10: wee. done with advance audit paper


'\n#The following code save SHAP analysis results into different html files for each tweet:\n\n#shap.save_html("shap_analysis.html", shap_values)\n# Manually set the base value if explainer.expected_value is None\n# For classification tasks, the base value can be the mean of logits\nif explainer.expected_value is None:\n    print("explainer.expected_value is None. Computing a manual base value.")\n    base_value = np.mean([np.mean(values.values) for values in shap_values])\nelse:\n    base_value = explainer.expected_value\n\n# Loop through all SHAP values and save each as a separate force plot\nfor i, shap_value in enumerate(shap_values):\n    # Debugging: Print base value and SHAP values\n    print(f"Processing Text {i + 1}")\n    print(f"Base Value: {base_value}")\n    print(f"SHAP Values for Text {i + 1}: {shap_value.values}")\n\n    # Check if shap_value.values is None\n    if shap_value.values is None:\n        print(f"Skipping Text {i + 1} due to missing values.")\n        contin